In [1]:
import numpy as np
from pyuvdata import UVData
import json
import astropy

In [ ]:
filename = '/lustre/aoc/projects/hera/dgorthi/H3C_IDR2/2458764/zen.2458764.60309.HH.uvh5'
uv = UVData()

In [ ]:
uv.read(filename)

In [ ]:
antpos = uv.get_ENU_antpos()
antpos_ENU = {k:v for k,v in zip(antpos[1],antpos[0])}
print(antpos_ENU)

In [2]:
with open('HERA_350_ENU.json','r') as fp:
    HERA_350_ENU = json.load(fp)

In [3]:
# Copied from the hera_mc database

cofa_lat = -30.72152612068925 * np.pi/180
cofa_lon = 21.42830382686301 * np.pi/180
cofa_alt = 1051.69

# Get speed of light
c = astropy.constants.c.value

# Hour Angle Range

The range of hour angles span from 8sec east of the zenith to 8sec west of zenith. There need to be 512 values between these two limits.

$$\frac{16s}{512} = 31.25 ms$$

That is, the beam is re-phased every 31.25 ms

In [4]:
ha0 = 8 / (24 * 60 * 60) * 2 * np.pi
print ('Radians: {0:.3f}\tDegrees: {1:.3f}'.format(ha0, ha0*180/np.pi))

Radians: 0.001	Degrees: 0.033


In [5]:
haf = -1 * 8 / (24 * 60 * 60) * 2 * np.pi
print ('Radians: {0:.3f}\tDegrees: {1:.3f}'.format(haf, haf*180/np.pi))

Radians: -0.001	Degrees: -0.033


In [6]:
ha_range = np.linspace(ha0, haf, num=512, endpoint=True)

# Time Delay 

Phasing to a hour angle that is not the zenith causes a geometric time delay between two antennas. Here the reference antenna is taken to the center of the array.

The time delay can be computed as follows (obtained from the undergraduate radio lab website):

$$\tau_g = \left[\frac{B_{ew}}{c} \cos{\delta}\right] \sin{H} + \left[\frac{B_{ns}}{c} \sin{\delta} \cos{L}\right] \cos{H} - \left[\frac{B_{ns}}{c} \cos{L} \sin{\delta}\right]$$

where $B_{ew}$ and $B_{ns}$ are the East-West and North-South components of the baseline vector, $H$ and $\delta$ are the hour angle and declination that we are phasing to and $L$ is the terrestrial latitude of the telescope.

The declination of the zenith is same as the terrestrial latitude of the telescope (nearly, say wiki).

Note: The latitute should probably be the center of the baseline vector but HERA spans only 300m of the 4e7m that the circumference of the Earth is. Is is 5e-5 radians of latitudinal extent. It possibly doesn't matter?? Using center of the array for now..

In [7]:
# Should latitute matter?? -- haven't thoroughly checked. The sine and cosine values 
# are changing in the 5th decimal place, so maybe?

print(np.sin(cofa_lat))
print(np.sin(cofa_lat + 300 * 2*np.pi/ 4e7))

print(np.cos(cofa_lat))
print(np.cos(cofa_lat + 300 * 2*np.pi/ 4e7))

-0.5108659298160229
-0.5108254187068926
0.85966039908397
0.859684472119235


In [8]:
geometric_delay_range = {}

for ant, loc in HERA_350_ENU.items():
    geometric_delay_range[ant] = loc[0]/c * np.cos(cofa_lat) * np.sin(ha_range) + \
                                 loc[1]/c * np.sin(cofa_lat) * np.cos(cofa_lat) * np.cos(ha_range) - \
                                 loc[1]/c * np.cos(cofa_lat) * np.sin(cofa_lat)

# Convert geometric delay into phase

The phase in terms of the geometric delay is given by:

$$\phi = 2\pi i \nu \tau_g$$

## FPGA Implementation

The FPGA design multiplies the input per-antenna phases by the channel number, for constructing the frequency dependence of the fringe stopping phases. Hence, the right input value to the FPGA is:

$$\phi = 2\pi i \Delta \nu \; \tau_g$$

so that:

$$\phi(\nu) = \text{chan} * 2 \pi i \Delta \nu \; \tau_g$$

The phase rotation is also implemented prior to channel downselect so all the 8192 (NOT just 6144) channels are present at this point.

## CORDIC 6.0

The block that finally implements the rotation requires the input in angle $\phi(\nu) \in [-\pi,\pi]$

In [9]:
chans = np.arange(8192)
nu = chans*(250e6/8192)
freqs = np.linspace(0,250,num=8192,endpoint=False)*1e6
np.all(np.isclose(freqs, nu))

True

In [10]:
# With 1536 channels finally
channel_width = 122070.3125

# With 6144 channels
channel_width = 250e6/8192.

In [11]:
# The FPGA design multiplies this
# input phase by channel number to 
# generate the rotation phases as
# a function of frequency

fringe_phase = {}

for ant, loc in HERA_350_ENU.items():
    fringe_phase[ant] = (2*np.pi*geometric_delay_range[ant]*channel_width).tolist()

In [14]:
np.asarray(fringe_phase[ant]) * 180/np.pi

array([ 3.58213773e-03,  3.56812179e-03,  3.55410581e-03,  3.54008980e-03,
        3.52607376e-03,  3.51205769e-03,  3.49804158e-03,  3.48402544e-03,
        3.47000927e-03,  3.45599306e-03,  3.44197682e-03,  3.42796055e-03,
        3.41394425e-03,  3.39992791e-03,  3.38591154e-03,  3.37189514e-03,
        3.35787871e-03,  3.34386224e-03,  3.32984574e-03,  3.31582921e-03,
        3.30181264e-03,  3.28779604e-03,  3.27377941e-03,  3.25976275e-03,
        3.24574605e-03,  3.23172932e-03,  3.21771256e-03,  3.20369577e-03,
        3.18967894e-03,  3.17566208e-03,  3.16164519e-03,  3.14762826e-03,
        3.13361130e-03,  3.11959431e-03,  3.10557729e-03,  3.09156023e-03,
        3.07754314e-03,  3.06352602e-03,  3.04950887e-03,  3.03549168e-03,
        3.02147446e-03,  3.00745721e-03,  2.99343992e-03,  2.97942260e-03,
        2.96540525e-03,  2.95138787e-03,  2.93737045e-03,  2.92335300e-03,
        2.90933552e-03,  2.89531801e-03,  2.88130046e-03,  2.86728288e-03,
        2.85326526e-03,  

In [15]:
with open('fringe_rotation_phases.json','w') as fp:
    json.dump(fringe_phase, fp)